In [23]:
!pip install keras tensorflow tf2onnx onnx


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
import os

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
import tf2onnx
import onnx

In [3]:
data = pd.read_csv('universal_top_spotify_songs.csv')

In [4]:
data.keys()

Index(['spotify_id', 'name', 'artists', 'daily_rank', 'daily_movement',
       'weekly_movement', 'country', 'snapshot_date', 'popularity',
       'is_explicit', 'duration_ms', 'album_name', 'album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'],
      dtype='object')

In [5]:
data = data.dropna()
mapping = {c:i for i, c in enumerate(data['country'].unique())}
pd.set_option('future.no_silent_downcasting', True)
data["country"] = data['country'].replace(
   mapping
).astype(int)

In [14]:
# Set the input (X) and output (Y) data. 
# The only output data is whether it's fraudulent. All other fields are inputs to the model.
X = data.drop(columns = ['spotify_id','name','artists', 'daily_rank', 'daily_movement', 'weekly_movement', 'country', 'snapshot_date', 'popularity', 'album_name', 'album_release_date', 'time_signature'])
y = np.asarray(data['country']).reshape(-1,1)

encoder = OneHotEncoder()
encoder.fit(y)
y_onehot = encoder.transform(y).toarray()

# Split the data into training and testing sets so you have something to test the trained model with.
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size = 0.2, shuffle = False)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2, stratify = y_train)

# Scale the data to remove mean and have unit variance. The data will be between -1 and 1, which makes it a lot easier for the model to learn than random (and potentially large) values.
# It is important to only fit the scaler to the training data, otherwise you are leaking information about the global distribution of variables (which is influenced by the test set) into the training set.
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.values)

In [52]:
Path("artifact").mkdir(parents=True, exist_ok=True)

with open("artifact/scaler.pkl", "wb") as handle:
    pickle.dump(scaler, handle)

In [17]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = len(X.columns)))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(len(mapping), activation = 'sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

/opt/app-root/lib64/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 32)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 72)             │        18,504 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,408 (243.78 KB)

 Trainable params: 62,408 (243.78 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
epochs = 2
history = model.fit(X_train, y_train, epochs=epochs, \
                    validation_data=(scaler.transform(X_val.values),y_val), \
                    verbose = True)
print("Training of model is complete")

Epoch 1/2
22322/22322 ━━━━━━━━━━━━━━━━━━━━ 23s 990us/step - accuracy: 0.1387 - loss: 3.4968 - val_accuracy: 0.3263 - val_loss: 2.5267
Epoch 2/2
22322/22322 ━━━━━━━━━━━━━━━━━━━━ 22s 993us/step - accuracy: 0.3584 - loss: 2.3773 - val_accuracy: 0.4047 - val_loss: 2.1582
Training of model is complete


In [26]:
model_proto, _ = tf2onnx.convert.from_keras(model)
os.makedirs("models/fraud/1", exist_ok=True)
onnx.save(model_proto, "models/fraud/1/model.onnx")

AttributeError: 'Sequential' object has no attribute '_get_save_spec'